Esse é o site que será a referência da ideia a ser desenvolvida aqui: 
https://github.com/conect2ai/descarbonizeai-co2-eq-models

# Preparando os Datasets

## IEEE Elétrico

https://ieee-dataport.org/open-access/battery-and-heating-data-real-driving-cycles

In [1]:
import os
import pandas as pd

In [2]:
directory = "./data/ieee/"
files = os.listdir(directory)
files.sort()

trips_ev = []
for file in files:
    trip = pd.read_csv(directory + file, sep=";", encoding='unicode_escape')
    trip = trip.sort_values(by='Time [s]')
    trip['Trip'] = file.split(".")[0]
    if 'Velocity [km/h]]]' in trip.columns:
        trip = trip.rename(columns={'Velocity [km/h]]]': 'Velocity [km/h]'})
    trips_ev.append(trip)

trips_ev = pd.concat(trips_ev)
trips_ev.head()

,Time [s],Velocity [km/h],Elevation [m],Throttle [%],Motor Torque [Nm],Longitudinal Acceleration [m/s^2],Regenerative Braking Signal,Battery Voltage [V],Battery Current [A],Battery Temperature [°C],...,Temperature Footweel Driver [°C],Temperature Footweel Co-Driver [°C],Temperature Feetvent Co-Driver [°C],Temperature Feetvent Driver [°C],Temperature Head Co-Driver [°C],Temperature Head Driver [°C],Temperature Vent right [°C],Temperature Vent central right [°C],Temperature Vent central left [°C],Temperature Vent right [°C]
0,0.0,0.0,574.0,0.0,0.0,-0.03,0.0,391.4,-2.20,21.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.1,0.0,574.0,0.0,0.0,0.00,0.0,391.4,-2.21,21.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.2,0.0,574.0,0.0,0.0,-0.01,0.0,391.4,-2.26,21.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.3,0.0,574.0,0.0,0.0,-0.03,0.0,391.4,-2.30,21.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.4,0.0,574.0,0.0,0.0,-0.03,0.0,391.4,-2.30,21.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
trips_ev.columns

Index(['Time [s]', 'Velocity [km/h]', 'Elevation [m]', 'Throttle [%]',
       'Motor Torque [Nm]', 'Longitudinal Acceleration [m/s^2]',
       'Regenerative Braking Signal ', 'Battery Voltage [V]',
       'Battery Current [A]', 'Battery Temperature [°C]',
       'max. Battery Temperature [°C]', 'SoC [%]', 'displayed SoC [%]',
       'min. SoC [%]', 'max. SoC [%)', 'Heating Power CAN [kW]',
       'Heating Power LIN [W]', 'Requested Heating Power [W]',
       'AirCon Power [kW]', 'Heater Signal', 'Heater Voltage [V]',
       'Heater Current [A]', 'Ambient Temperature [°C]',
       'Coolant Temperature Heatercore [°C]',
       'Requested Coolant Temperature [°C]', 'Coolant Temperature Inlet [°C]',
       'Heat Exchanger Temperature [°C]', 'Cabin Temperature Sensor [°C]',
       'Trip', 'Unnamed: 23', 'Ambient Temperature Sensor [°C]',
       'Coolant Volume Flow +500 [l/h]',
       'Temperature Coolant Heater Inlet [°C]',
       'Temperature Coolant Heater Outlet [°C]',
       'Temperatu

In [4]:
trips_ev['Battery Voltage [V]'].isnull().any() and trips_ev['Battery Current [A]'].isnull().any()

np.False_

In [5]:
trips_ev['CO2 Emissions'] = (abs(trips_ev['Battery Current [A]'] * trips_ev['Battery Voltage [V]']) / 1000) * 38.5
trips_ev['CO2 Emissions'] 

0        33.151580
1        33.302269
2        34.055714
3        34.658470
4        34.658470
           ...    
16424    56.541100
16425    59.376240
16426    57.247864
16427    55.120065
16428    55.120065
Name: CO2 Emissions, Length: 1094793, dtype: float64

In [6]:
colunas_selecionadas = [
    'Time [s]', 
    'Velocity [km/h]',
    'Throttle [%]',
    'Motor Torque [Nm]',
    'CO2 Emissions',
    'Trip'
]
trips_ev = trips_ev[colunas_selecionadas]
trips_ev.head()

,Time [s],Velocity [km/h],Throttle [%],Motor Torque [Nm],CO2 Emissions,Trip
0,0.0,0.0,0.0,0.0,33.151580,TripA01
1,0.1,0.0,0.0,0.0,33.302269,TripA01
2,0.2,0.0,0.0,0.0,34.055714,TripA01
3,0.3,0.0,0.0,0.0,34.658470,TripA01
4,0.4,0.0,0.0,0.0,34.658470,TripA01


## Argonne Combustão

https://www.anl.gov/taps/electric-vehicle-testing

In [8]:
trips_ice = pd.read_csv("./data/EquivalenteCombustaoDanielNovo.csv")
trips_ice.head()

,DAQ_Time,Time,Dyno_Spd,Dyno_TractiveForce,Dyno_LoadCell,Distance,Dyno_Spd_Front,Dyno_TractiveForce_Front,Dyno_LoadCell_Front,Dyno_Spd_Rear,...,TrnsTrqCnvCltDsrSlp,ThrtCommndPos,ThrPos,EngManfldAbsPrs,FlInjRlCt,TrnGrRat,EngCstFlCutAct,ACCompCmnd,Eng_fuel_FFM_Mass_Flowrate_FFM_1,Eng_fuel_FFM_Volume_Flowrate_FFM_1
0,224.780324,-20.0,0.000000,-11.930436,-7.131023,7.465411e-07,0.000000,-9.239001,1.771972,0.0,...,32.140791,0.0,9.096202,45.695307,0.006287,7.96875,0.0,0.0,0.062202,0.082960
1,224.880944,-19.9,0.000028,-11.981836,-0.961913,7.473089e-07,0.000028,-9.290401,4.940838,0.0,...,12.482216,0.0,9.411768,46.500000,0.006287,7.96875,0.0,0.0,-1.713110,-2.282712
2,224.981114,-19.8,0.000006,-12.043709,-11.983204,7.474897e-07,0.000006,-9.334732,-3.030165,0.0,...,30.219152,0.0,9.335214,46.304788,0.006304,7.96875,0.0,0.0,-0.755663,-1.006851
3,225.081119,-19.7,0.000000,-12.007828,-11.035185,7.474897e-07,0.000000,-9.312310,-0.888739,0.0,...,21.976490,0.0,9.095380,45.789818,0.006378,7.96875,0.0,0.0,1.801878,2.401025
4,225.181130,-19.6,0.000028,-12.002788,-0.502919,7.482593e-07,0.000028,-9.311353,5.209817,0.0,...,7.156888,0.0,9.411768,46.906714,0.006412,7.96875,0.0,0.0,2.104205,2.803601


In [9]:
trips_ice.columns

Index(['DAQ_Time', 'Time', 'Dyno_Spd', 'Dyno_TractiveForce', 'Dyno_LoadCell',
       'Distance', 'Dyno_Spd_Front', 'Dyno_TractiveForce_Front',
       'Dyno_LoadCell_Front', 'Dyno_Spd_Rear',
       ...
       'TrnsTrqCnvCltDsrSlp', 'ThrtCommndPos', 'ThrPos', 'EngManfldAbsPrs',
       'FlInjRlCt', 'TrnGrRat', 'EngCstFlCutAct', 'ACCompCmnd',
       'Eng_fuel_FFM_Mass_Flowrate_FFM_1',
       'Eng_fuel_FFM_Volume_Flowrate_FFM_1'],
      dtype='object', length=187)

In [10]:
# PAREI AQUI - Preciso montar o dataset para ficar com as mesmas colunas do IEEE Elétrico que eu já preparei.
# A fórmula de emissões de CO2 tem no git e na conversa com o Rodrigo